# Evaluation of MSSW on SEA

## Imports

In [10]:
import numpy as np
import pandas as pd
import sklearn

## SEA dataset locations

In [11]:
abrupt_sea_path = '../Datasets_concept_drift/synthetic_data/abrupt_drift/sea_1_abrupt_drift_0_noise_balanced.arff'
gradual_sea_paths = [
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_05.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_1.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_5.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_10.arff',
    '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_20.arff'
]
all_sea_data_paths = [abrupt_sea_path] + gradual_sea_paths

## Accept and preprocess SEA datasets

In [ ]:
import accepting
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


sea_reference_batches = {}
sea_testing_batches = {}
for sea_path in all_sea_data_paths:
    df_x, df_y = accepting.get_clean_df(sea_path)
    df_y = pd.DataFrame(LabelEncoder().fit_transform(df_y))

    df_x_ref, df_x_test, df_y_ref, df_y_test = sklearn.model_selection.train_test_split(
        df_x, df_y, test_size=0.7, shuffle=False)
    
    reference_data = df_x_ref.to_numpy()
    testing_data = df_x_test.to_numpy()
    num_ref_batches = 3
    num_test_batches = 7
    ref_batches = np.array_split(reference_data, num_ref_batches)
    test_batches = np.array_split(testing_data, num_test_batches)
    
    sea_reference_batches[sea_path] = ref_batches
    sea_testing_batches[sea_path] = test_batches

df         attrib1   attrib2   attrib3      class
0      7.308782  4.100808  2.077148  b'groupB'
1      5.833539  0.422983  7.616747  b'groupA'
2      1.397627  6.949480  8.052278  b'groupB'
3      2.750299  0.753878  6.105915  b'groupA'
4      2.049135  6.233638  1.847071  b'groupB'
...         ...       ...       ...        ...
99995  4.636430  6.639370  0.066740  b'groupB'
99996  4.251993  3.351235  5.652197  b'groupA'
99997  4.131405  6.371722  3.125554  b'groupB'
99998  1.404214  4.392506  9.298558  b'groupA'
99999  7.231749  8.770465  3.925490  b'groupB'

[100000 rows x 4 columns]


C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


df         attrib1   attrib2   attrib3      class
0      7.308782  4.100808  2.077148  b'groupB'
1      5.833539  0.422983  7.616747  b'groupA'
2      1.397627  6.949480  8.052278  b'groupB'
3      2.750299  0.753878  6.105915  b'groupA'
4      2.049135  6.233638  1.847071  b'groupB'
...         ...       ...       ...        ...
99995  2.074508  1.775662  1.318589  b'groupA'
99996  4.636430  6.639370  0.066740  b'groupB'
99997  4.251993  3.351235  5.652197  b'groupA'
99998  4.131405  6.371722  3.125554  b'groupB'
99999  1.404214  4.392506  9.298558  b'groupA'

[100000 rows x 4 columns]


C:\Users\jpohl\PycharmProjects\clustering-drift-detection\mssw\accepting.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.decode('utf-8')
c:\users\jpohl\pycharmprojects\clustering-drift-detection\venv\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Find the best tol and max_iter in SEA (the drift type is irrelevant)

In [5]:
import mssw_preprocessing
import kmeans_verbose_helpers


weighted_joined_reference_data, _, _ = mssw_preprocessing.mssw_preprocess(
    sea_reference_batches[abrupt_sea_path], sea_testing_batches[abrupt_sea_path])

filename = 'sea_new_output.txt'
kmeans_verbose_helpers.write_verbose_kmeans_to_file(filename, weighted_joined_reference_data,
                             n_clusters=2, n_init=100, max_iter=500, tol=0, random_state=1053)
output_dicts = kmeans_verbose_helpers.convert_kmeans_output_file_to_dicts(filename, n_init=100)
kmeans_verbose_helpers.print_stats_from_kmeans_output_dicts(output_dicts)

random state: 1053
total number of results: 100
maximum number of iterations: 54
minimum initial inertia: 657.5117945560726
maximum initial inertia: 1654.92779186788
number of unique final inertia values: 17
minimum final inertia: 597.8461904607495
maximum final inertia: 605.4169667607526
total number of convergences: 100
number of strict convergences: 100
number of tol-based convergences: 0


## Use them for the analysis

In [6]:
import mssw_eval



sea_stats = {}
for sea_path in all_sea_data_paths:
    runs_results_bool, final_fpr_mean, fpr_std_err, final_latency_mean, latency_std_err = \
        mssw_eval.all_drifting_batches_randomness_robust(
        sea_reference_batches[sea_path],
        sea_testing_batches[sea_path],
        n_clusters=2,
        n_init=100,
        max_iter=550,
        tol=0,
        true_drift_idx=2,
        min_runs=2
    )
    sea_stats[sea_path] = {
        'runs_results_bool': runs_results_bool,
        'final_fpr_mean': final_fpr_mean,
        'fpr_std_err': fpr_std_err,
        'final_latency_mean': final_latency_mean,
        'latency_std_err': latency_std_err
    }

print('SEA STATS')
print(sea_stats)

min_runs 2
centroid distance sums [[ 945.67719668 1356.42528063]]
num points in clusters [[3795. 6205.]]
centroid distance sums [[ 950.35491978 1335.33354816]]
num points in clusters [[3864. 6136.]]
centroid distance sums [[ 934.83093979 1350.55732975]]
num points in clusters [[3792. 6208.]]
centroid distance sums [[ 948.51413223 1343.04761502]]
num points in clusters [[3836. 6164.]]
centroid distance sums [[ 932.50752815 1362.37095108]]
num points in clusters [[3753. 6247.]]
centroid distance sums [[1095.37007132 1238.4956423 ]]
num points in clusters [[4421. 5579.]]
centroid distance sums [[1224.4535414  1169.54152057]]
num points in clusters [[4921. 5079.]]
centroid distance sums [[1216.65097587 1166.11643499]]
num points in clusters [[4938. 5062.]]
centroid distance sums [[1220.59723706 1160.67009289]]
num points in clusters [[4911. 5089.]]
centroid distance sums [[1232.38081469 1156.44808206]]
num points in clusters [[4955. 5045.]]
centroid distance sums [[1357.13848147  944.95695

KeyError: '../Datasets_concept_drift/synthetic_data/gradual_drift/sea_1_gradual_drift_0_noise_balanced_5.arff'

In [9]:
import helpers


final_result_dict = {
    'type_of_data': [], 'dataset': [], 'drift': [], 'width': [], 'encoding': [],
    'n_init': [], 'max_iter': [], 'tol': [],
    'FPR_mean': [], 'latency_mean': []
}

for data_path, stats_dict in sea_stats.items():
#     data_filename = data_path.split('/')[-1]
#     type_of_data = data_path.split('/')[2].split('_')[0]  # synthetic or real-world
#     dataset_name = data_filename.split('_')[0]  # sea, agraw1, agraw2
#     drift_type = data_path.split('/')[3].split('_')[0]
#     drift_width = '0' if drift_type == 'abrupt' else data_filename.split('_')[-1].split('.')[0]
#     drift_width = 0.5 if drift_width == '05' else float(drift_width)
    synthetic_filename_info = helpers.synthetic_data_information(data_path)
    encoding = 'exclude'
    fpr_mean = float(stats_dict['final_fpr_mean'])
    latency_mean = float(stats_dict['final_latency_mean'])
    
    final_result_dict['type_of_data'].append(synthetic_filename_info['type_of_data'])
    final_result_dict['dataset'].append(synthetic_filename_info['dataset_name'])
    final_result_dict['drift'].append(synthetic_filename_info['drift_type'])
    final_result_dict['width'].append(synthetic_filename_info['drift_width'])
    final_result_dict['encoding'].append(encoding)
    final_result_dict['n_init'].append(100)
    final_result_dict['max_iter'].append(550)
    final_result_dict['tol'].append(0)
    final_result_dict['FPR_mean'].append(fpr_mean)
    final_result_dict['latency_mean'].append(latency_mean)
    
final_result_df = pd.DataFrame.from_dict(final_result_dict)
sorted_final_result_df = final_result_df.sort_values(['drift', 'dataset', 'encoding', 'width'])
final_result_df.to_csv('sea_jupyter_results2.csv', index=False)